In [6]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Coding Exercise 1: Plotting a histogram

# Set a random number generator
np.random.seed(2020)

# Initialize t_range, step_end, n, v_n, i, and nbins
t_range = 
step_end = 
n = 10000
v_n = 
i = 
nbins = 50